# Variables
* Binary variable A(h, n) for h=1to5, for n=1to4
* Binary variable B(h, n) for h=1to5, for n=0to3
* Binary variable C(h, n) for h=1to5, for n=0to4
* Binary variable D(h, n) for h=1to5, for n=0to1
* Integer variable N(h) for h=1to5 

# Constraints
## Variable bounds
* all A, B, C, D between 0 and 1
* N between 0 and 170 

## Consistency / Only one possible choice for how many you assign
* SUM(A(h, n) for n=1to4) == 1, for h=1to5
* SUM(B(h, n) for n=0to3) == 1, for h=1to5
* SUM(C(h, n) for n=0to4) == 1, for h=1to5
* SUM(D(h, n) for n=0to1) == 1, for h=1to5

## Maximum total specialist
* SUM(n*A(h, n) for h=1to5, for n=1to4) <= 8
* SUM(n*B(h, n) for h=1to5, for n=0to3) <= 3
* SUM(n*C(h, n) for h=1to5, for n=0to4) <= 4
* SUM(n*D(h, n) for h=1to5, for n=0to1) <= 3
* SUM(N(h) for h=1to5) <= 250

## At least one of type B OR C
* B(h, 0) + C(h, 0) <=1 , for h=1to5


# Objective Function
MAXIMIZE
SUM(
A(h, 1) * 0 + A(h, 2) * 7 + A(h, 3) * 11 + A(h, 4) * 14
+ B(h, 0) * 0 + ...
+ C(h, 0) * 0 + ...
+ D(h, 0) * 0 + ...
+ N(h)
) for h=1to5

In [ ]:
N(1) = 20
N(2) = 30

benefit = N(1) + N(2) + ...

In [97]:
import pulp as plp

In [98]:
specialists = 5
hospitals = 5

In [99]:
range_A = range(1,5)
range_B = range(0,4)
range_B_except_first = range(1,4)
range_C = range(0,5)
range_C_except_first = range(1,5)
range_D = range(0,2)
range_H = range(1,hospitals+1)

In [100]:
benefits_A = [0, 7, 11, 14, 16, 18, 19, 21]
benefits_B = [0, 1, 2, 5]
benefits_C = [0, 0, 1, 3, 5]
benefits_D = [0, 2]

In [112]:
opt_model = None
opt_model = plp.LpProblem(name='Hospital')
opt_model

# variables
#     variable tracking the assignment of each type to each hospital
A = {(h,n): plp.LpVariable(cat='Binary', name="A_{0}_{1}".format(h,n))
                             for h in range_H for n in range_A}
B = {(h,n): plp.LpVariable(cat='Binary', name="B_{0}_{1}".format(h,n))
                             for h in range_H for n in range_B}
C = {(h,n): plp.LpVariable(cat='Binary', name="C_{0}_{1}".format(h,n))
                             for h in range_H for n in range_C}
D = {(h,n): plp.LpVariable(cat='Binary', name="D_{0}_{1}".format(h,n))
                             for h in range_H for n in range_D}
N = {(h): plp.LpVariable(cat='Integer', name="N_{0}".format(h), lowBound=20, upBound=250) for h in range_H}

# constraints
for h in range_H:
# at least one of B or C
#     opt_model += plp.lpSum(B[h,n] for n in range_B_except_first) + plp.lpSum(C[h,n] for n in range_C_except_first) >=1, "1_type_BorC_min_for_H{0}".format(h)
    opt_model += B[h,0] + C[h,0] <= 1, "1_type_BorC_min_for_H{0}".format(h)

# all binaries for same specialist are exclusive
    opt_model += plp.lpSum(A[h,n] for n in range_A)==1, "only_1_bin_A_for_H{0}".format(h)
    opt_model += plp.lpSum(B[h,n] for n in range_B)==1, "only_1_bin_B_for_H{0}".format(h)
    opt_model += plp.lpSum(C[h,n] for n in range_C)==1, "only_1_bin_C_for_H{0}".format(h)
    opt_model += plp.lpSum(D[h,n] for n in range_D)==1, "only_1_bin_D_for_H{0}".format(h)

# sum accross all binaries*their count value of 
# same specialist across all hospitals is smaller than maximum
opt_model += plp.lpSum(n*A[h,n] for n in range_A for h in range_H)<=8, "limit_on_A"
opt_model += plp.lpSum(n*B[h,n] for n in range_B for h in range_H)<=3, "limit_on_B"
opt_model += plp.lpSum(n*C[h,n] for n in range_C for h in range_H)<=4, "limit_on_C"
opt_model += plp.lpSum(n*D[h,n] for n in range_D for h in range_H)<=3, "limit_on_D"
opt_model += plp.lpSum(N[h] for h in range_H)<=250, "limit_on_N"



# objective
objective = plp.lpSum(N[h] for h in range_H)
objective += plp.lpSum(benefits_A[n-1]*A[h,n] for n in range_A for h in range_H)
objective += plp.lpSum(benefits_B[n]*B[h,n] for n in range_B for h in range_H)
objective += plp.lpSum(benefits_C[n]*C[h,n] for n in range_C for h in range_H)
objective += plp.lpSum(benefits_D[n]*D[h,n] for n in range_D for h in range_H)

opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)

opt_model.writeLP('model_new.lp')

opt_model.solve()
print("Model {} status: {}".format(opt_model.name, plp.LpStatus[opt_model.status]))
solution = {}
for var in opt_model.variables():
    solution[var.name] = var.value()

Model Hospital status: Optimal


In [111]:
for h in range_H:
    print('\n Hospital', h)
    for n in range_A:
        if solution['A_'+str(h)+'_'+str(n)]>=1:
            print(n,'type A specialists')
    for n in range_B:
        if solution['B_'+str(h)+'_'+str(n)]>=1:
            print(n,'type B specialists')
    for n in range_C:
        if solution['C_'+str(h)+'_'+str(n)]>=1:
            print(n,'type C specialists')
    for n in range_D:
        if solution['D_'+str(h)+'_'+str(n)]>=1:
            print(n,'type D specialists')
    print(solution['N_'+str(h)],'Nurse Practitioners')



 Hospital 1
2 type A specialists
0 type B specialists
3 type C specialists
1 type D specialists
20.0 Nurse Practitioners

 Hospital 2
1 type A specialists
1 type B specialists
0 type C specialists
0 type D specialists
20.0 Nurse Practitioners

 Hospital 3
2 type A specialists
1 type B specialists
0 type C specialists
0 type D specialists
20.0 Nurse Practitioners

 Hospital 4
2 type A specialists
1 type B specialists
0 type C specialists
1 type D specialists
20.0 Nurse Practitioners

 Hospital 5
1 type A specialists
0 type B specialists
1 type C specialists
1 type D specialists
170.0 Nurse Practitioners


In [109]:
plp.value(opt_model.objective)

283.0

# EXTENSION B

## New Variable
* Binary variable E(h, n) for h=1to5, for n=0to5

# New Constraint
## Consistency
* SUM(E(h, n) for n=0to1) == 1, for h=1to5

## Maximum total specialist E
* SUM(n*E(h, n) for h=1to5, for n=0to5) <= 5

## Only E if type C assigned
* SUM(E(h, n) for n=1to5) <= 1-C(h, 0), for h=1to5

# Objective 
 * ORIGINAL OBJECTIVE + E(h, 0) * 0 + ...

# EXTENSION C

MAXIMIZE
    ORIGINAL OBJECTIVE +
    N(h)*B(h,1) + N(h)*B(h,2)+ N(h)*B(h,3) +N(h)*B(h,4)
    
Becomes non-linear, let's abandon it